In [7]:
import os
import requests
from Arxiv_Parser.paper_parser import parse_html
from Arxiv_Parser.paper_storage import save_paper_data
from LLM.llm import MultiLLM
from Task_Conductor.prompts import RelevanceTask

# 新增默认配置常量
DEFAULT_SAVE_DIR = "./papers/"  # 默认存储目录
DEFAULT_FILENAME_TEMPLATE = "paper_{arxiv_id}.json"  # 默认文件名模板

def process_paper(url, output_path=None, keyword=None):
    """
    改进版论文处理流程（带默认路径）
    参数：
    url - arXiv HTML页面地址
    output_path - 可以是目录/文件路径（可选，默认存储在./papers/）
    keyword - 相关性分析关键词（可选）
    """
    try:
        # 生成arXiv ID
        arxiv_id = url.split("/")[-1].replace("v1", "").replace("html/", "")
        
        # 生成默认文件名
        default_filename = DEFAULT_FILENAME_TEMPLATE.format(arxiv_id=arxiv_id)
        
        # 处理输出路径逻辑
        if output_path:
            # 用户指定路径处理
            if os.path.isdir(output_path) or output_path.endswith(('/','\\')):
                # 用户指定的是目录
                output_path = os.path.join(output_path, default_filename)
        else:
            # 使用默认存储目录
            output_path = os.path.join(DEFAULT_SAVE_DIR, default_filename)
        
        # 确保存储目录存在
        os.makedirs(os.path.dirname(output_path), exist_ok=True)
        
        # 获取论文内容
        print("🔄 正在获取论文内容...")
        response = requests.get(url)
        response.raise_for_status()
        
        # 解析论文
        print("🔍 解析论文结构...")
        paper_data = parse_html(response.text)
        
        # 保存结果
        print(f"💾 保存到：{output_path}")
        save_paper_data(paper_data, output_path)
        
        # 关键词分析（保持原逻辑）
        if keyword:
            print(f"🔎 正在分析关键词 '{keyword}' 相关性...")
            llm = MultiLLM('deepseek-coder')
            task = RelevanceTask(paper_data["abstract"], keyword)
            
            response = llm.ask(task.generate_prompt())
            score = task.parse_model_output(response)
            
            if score:
                print(f"\n⭐ 相关性评分：{score}/1.0")
        
        print("✅ 处理完成！")
        return paper_data
        
    except Exception as e:
        print(f"❌ 处理失败：{str(e)}")
        raise


In [8]:
# 直接执行单个论文分析
paper_data = process_paper(
    url="https://arxiv.org/html/2501.00092v1",
    keyword="quantum"  # 可选参数
)

# 查看结果结构
print(f"标题：{paper_data['title']}")
print(f"摘要：{paper_data['abstract'][:200]}...")


🔄 正在获取论文内容...
🔍 解析论文结构...
💾 保存到：./papers/paper_2501.00092.json
论文数据成功保存到 ./papers/paper_2501.00092.json
🔎 正在分析关键词 'quantum' 相关性...

⭐ 相关性评分：0.3/1.0
✅ 处理完成！
标题：Moments and saddles of heavy CFT correlators
摘要：
Abstract
We study the operator product expansion (OPE) of identical scalars in a conformal four-point correlator as a Stieltjes moment problem, and use Riemann-Liouville type fractional differential ...


In [9]:
{'title': '...', 'abstract': '...', 'sections': [{'title': '...', 'id': None, 'subsections': [{'title': '...', 'id': 'S1', 'subsections': [], 'text': '...'}, {'title': '...', 'id': 'S2', 'subsections': [{'title': '...', 'id': '...', 'subsections': [], 'text': '...'}, {'title': '...', 'id': '...', 'subsections': [], 'text': '...'}], 'text': '...'}, {'title': '...', 'id': 'S3', 'subsections': [{'title': '...', 'id': '...', 'subsections': [{'title': '...', 'id': '...', 'subsections': [], 'text': '...'}], 'text': '...'}, {'title': '...', 'id': '...', 'subsections': [], 'text': '...'}, {'title': '...', 'id': '...', 'subsections': [{'title': '...', 'id': '...', 'subsections': [], 'text': '...'}, {'title': '...', 'id': '...', 'subsections': [], 'text': '...'}], 'text': '...'}], 'text': '...'}, {'title': '...', 'id': 'S4', 'subsections': [{'title': '...', 'id': '...', 'subsections': [], 'text': '...'}, {'title': '...', 'id': '...', 'subsections': [], 'text': '...'}, {'title': '...', 'id': '...', 'subsections': [], 'text': '...'}], 'text': '...'}, {'title': '...', 'id': 'S5', 'subsections': [{'title': '...', 'id': '...', 'subsections': [], 'text': '...'}, {'title': '...', 'id': '...', 'subsections': [], 'text': '...'}], 'text': '...'}, {'title': '...', 'id': 'S6', 'subsections': [], 'text': '...'}, {'title': '...', 'id': 'A1', 'subsections': [], 'text': '...'}, {'title': '...', 'id': 'A2', 'subsections': [], 'text': '...'}]}], 'references': {'bib.bib1': {'meta_list': ['...'], 'meta_string': '...', 'authors': 'S.', 'title': '...', 'journal': '...'}, 'bib.bib2': {'meta_list': ['...'], 'meta_string': '...', 'authors': '...', 'title': '...', 'journal': '...'}, 'bib.bib3': {'meta_list': ['...'], 'meta_string': '...', 'authors': 'D.', 'title': '...', 'journal': '...'}, 'bib.bib4': {'meta_list': ['...'], 'meta_string': '...', 'authors': '...', 'title': '...', 'journal': '...'}, 'bib.bib5': {'meta_list': ['...'], 'meta_string': '...', 'authors': '...', 'title': '...', 'journal': '...'}, 'bib.bib6': {'meta_list': ['...'], 'meta_string': '...', 'authors': 'E.', 'title': '...', 'journal': '...'}, 'bib.bib7': {'meta_list': ['...'], 'meta_string': '...', 'authors': '', 'title': '', 'journal': ''}, 'bib.bib8': {'meta_list': ['...'], 'meta_string': '...', 'authors': 'S.', 'title': '...', 'journal': '...'}, 'bib.bib9': {'meta_list': ['...'], 'meta_string': '...', 'authors': 'S.', 'title': '...', 'journal': '...'}, 'bib.bib10': {'meta_list': ['...'], 'meta_string': '...', 'authors': 'S.', 'title': '...', 'journal': '...'}, 'bib.bib11': {'meta_list': ['...'], 'meta_string': '...', 'authors': 'F.', 'title': '...', 'journal': '...'}, 'bib.bib12': {'meta_list': ['...'], 'meta_string': '...', 'authors': '...', 'title': '...', 'journal': '...'}, 'bib.bib13': {'meta_list': ['...'], 'meta_string': '...', 'authors': 'F.', 'title': '...', 'journal': '...'}, 'bib.bib14': {'meta_list': ['...'], 'meta_string': '...', 'authors': '...', 'title': '...', 'journal': '...'}, 'bib.bib15': {'meta_list': ['...'], 'meta_string': '...', 'authors': '...', 'title': '...', 'journal': '...'}, 'bib.bib16': {'meta_list': ['...'], 'meta_string': '...', 'authors': 'A.', 'title': '...', 'journal': '...'}, 'bib.bib17': {'meta_list': ['...'], 'meta_string': '...', 'authors': '...', 'title': '...', 'journal': '...'}, 'bib.bib18': {'meta_list': ['...'], 'meta_string': '...', 'authors': 'S.', 'title': '...', 'journal': '...'}, 'bib.bib19': {'meta_list': ['...'], 'meta_string': '...', 'authors': '...', 'title': '...', 'journal': '...'}, 'bib.bib20': {'meta_list': ['...'], 'meta_string': '...', 'authors': 'A.', 'title': '...', 'journal': ''}, 'bib.bib21': {'meta_list': ['...'], 'meta_string': '...', 'authors': 'G.', 'title': '...', 'journal': '...'}, 'bib.bib22': {'meta_list': ['...'], 'meta_string': '...', 'authors': 'J.', 'title': '...', 'journal': '...'}, 'bib.bib23': {'meta_list': ['...'], 'meta_string': '...', 'authors': 'D.', 'title': '...', 'journal': ''}, 'bib.bib24': {'meta_list': ['...'], 'meta_string': '...', 'authors': 'L.', 'title': '...', 'journal': '...'}, 'bib.bib25': {'meta_list': ['...'], 'meta_string': '...', 'authors': 'N.', 'title': '...', 'journal': '...'}, 'bib.bib26': {'meta_list': ['...'], 'meta_string': '...', 'authors': 'H.', 'title': '...', 'journal': '...'}, 'bib.bib27': {'meta_list': ['...'], 'meta_string': '...', 'authors': '...', 'title': '...', 'journal': '...'}, 'bib.bib28': {'meta_list': ['...'], 'meta_string': '...', 'authors': '...', 'title': '...', 'journal': '...'}, 'bib.bib29': {'meta_list': ['...'], 'meta_string': '...', 'authors': 'P.', 'title': '...', 'journal': '...'}, 'bib.bib30': {'meta_list': ['...'], 'meta_string': '...', 'authors': 'P.', 'title': '...', 'journal': '...'}, 'bib.bib31': {'meta_list': ['...'], 'meta_string': '...', 'authors': '...', 'title': '...', 'journal': '...'}, 'bib.bib32': {'meta_list': ['...'], 'meta_string': '...', 'authors': '...', 'title': '...', 'journal': '...'}, 'bib.bib33': {'meta_list': ['...'], 'meta_string': '...', 'authors': 'M.', 'title': '...', 'journal': '...'}, 'bib.bib34': {'meta_list': ['...'], 'meta_string': '...', 'authors': 'J.', 'title': '...', 'journal': '...'}, 'bib.bib35': {'meta_list': ['...'], 'meta_string': '...', 'authors': 'C.', 'title': '...', 'journal': ''}, 'bib.bib36': {'meta_list': ['...'], 'meta_string': '...', 'authors': '...', 'title': '...', 'journal': ''}, 'bib.bib37': {'meta_list': ['...'], 'meta_string': '...', 'authors': '...', 'title': '...', 'journal': '...'}, 'bib.bib38': {'meta_list': ['...'], 'meta_string': '...', 'authors': 'K.', 'title': '...', 'journal': ''}, 'bib.bib39': {'meta_list': ['...'], 'meta_string': '...', 'authors': 'K.', 'title': '...', 'journal': ''}, 'bib.bib40': {'meta_list': ['...'], 'meta_string': '...', 'authors': '...', 'title': '...', 'journal': '...'}, 'bib.bib41': {'meta_list': ['...'], 'meta_string': '...', 'authors': 'Z.', 'title': '...', 'journal': '...'}, 'bib.bib42': {'meta_list': ['...'], 'meta_string': '...', 'authors': 'S.', 'title': '...', 'journal': '...'}, 'bib.bib43': {'meta_list': ['...'], 'meta_string': '...', 'authors': '...', 'title': '...', 'journal': ''}, 'bib.bib44': {'meta_list': ['...'], 'meta_string': '...', 'authors': 'M.', 'title': '...', 'journal': '...'}, 'bib.bib45': {'meta_list': ['...'], 'meta_string': '...', 'authors': 'I.', 'title': '...', 'journal': '...'}, 'bib.bib46': {'meta_list': ['...'], 'meta_string': '...', 'authors': 'L.', 'title': '...', 'journal': '...'}, 'bib.bib47': {'meta_list': ['...'], 'meta_string': '...', 'authors': 'W.', 'title': '...', 'journal': '...'}, 'bib.bib48': {'meta_list': ['...'], 'meta_string': '...', 'authors': '...', 'title': '...', 'journal': '...'}, 'bib.bib49': {'meta_list': ['...'], 'meta_string': '...', 'authors': 'P.', 'title': '...', 'journal': '...'}, 'bib.bib50': {'meta_list': ['...'], 'meta_string': '...', 'authors': 'P.', 'title': '...', 'journal': '...'}, 'bib.bib51': {'meta_list': ['...'], 'meta_string': '...', 'authors': 'K.', 'title': '...', 'journal': '...'}, 'bib.bib52': {'meta_list': ['...'], 'meta_string': '...', 'authors': '...', 'title': '...', 'journal': '...'}, 'bib.bib53': {'meta_list': ['...'], 'meta_string': '...', 'authors': 'K.', 'title': '...', 'journal': '...'}, 'bib.bib54': {'meta_list': ['...'], 'meta_string': '...', 'authors': 'S.', 'title': '...', 'journal': '...'}}}

{'title': '...',
 'abstract': '...',
 'sections': [{'title': '...',
   'id': None,
   'subsections': [{'title': '...',
     'id': 'S1',
     'subsections': [],
     'text': '...'},
    {'title': '...',
     'id': 'S2',
     'subsections': [{'title': '...',
       'id': '...',
       'subsections': [],
       'text': '...'},
      {'title': '...', 'id': '...', 'subsections': [], 'text': '...'}],
     'text': '...'},
    {'title': '...',
     'id': 'S3',
     'subsections': [{'title': '...',
       'id': '...',
       'subsections': [{'title': '...',
         'id': '...',
         'subsections': [],
         'text': '...'}],
       'text': '...'},
      {'title': '...', 'id': '...', 'subsections': [], 'text': '...'},
      {'title': '...',
       'id': '...',
       'subsections': [{'title': '...',
         'id': '...',
         'subsections': [],
         'text': '...'},
        {'title': '...', 'id': '...', 'subsections': [], 'text': '...'}],
       'text': '...'}],
     'text': '...'}